# Tutorial 3: Multiagent scenarios with PettingZoo 

In this tutorial, we will use Cogment Lab to interact with a multiagent environment from the [PettingZoo](https://www.pettingzoo.farama.org/) library. We will combine simple algorithmic agents with a human player to play a game of cooperative Pong.

Most of the code is the same as with Gymnasium -- check out the previous tutorial to understand it.

In [1]:
from cogment_lab import Cogment
from cogment_lab.envs import ParallelEnvironment 
from cogment_lab.actors import RandomActor, ConstantActor

In [2]:
cog = Cogment(log_dir="logs/tutorial3")

In [3]:
cenv = ParallelEnvironment(
  env_path="pettingzoo.butterfly.cooperative_pong_v5.parallel_env",
  render=True
)

await cog.run_env(env=cenv, 
                  env_name="pong",
                  port=9011, 
                  log_file="env.log")

True

In [4]:
act_space = cenv.env.action_space("paddle_0")

In [5]:
actor = RandomActor(action_space=act_space)

await cog.run_actor(actor, 
  actor_name="random",
  port=9021,
  log_file="actor-random.log"
)

True

In [6]:
constant_actor = ConstantActor(action=0)

await cog.run_actor(constant_actor, 
  actor_name="constant",
  port=9022,
  log_file="actor-constant.log"
)

True

Just like with Gymnasium, we can get a trial from a PettingZoo environment by specifying the environment and the actors -- we just need to specify an actor name for each agent in the environment.

In [7]:
cenv.env.possible_agents

['paddle_0', 'paddle_1']

In [8]:
trial_id = await cog.start_trial(
    env_name="pong",  # Name of the environment to use
    actor_impls={
        "paddle_0": "random",
        "paddle_1": "constant"
    },
)
data = await cog.get_trial_data(trial_id)

In [9]:
print(f"Hopefully random actions: {data['paddle_0'].actions}")
print(f"Hopefully constant actions: {data['paddle_1'].actions}")

Hopefully random actions: [1 1 0 1 0 1 1 2 1 0 2 0 1 1 2 1 0 0 2 1 1 2 1 1 2 1 0 2 1 2 1 1 1 0 1 0 2
 0 0 0 0 1 0 2 2 0 0 2]
Hopefully constant actions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]


Just like with Gymnasium, we can use a human in the trial. Unlike Gymnasium, we can have both a human and an "AI" acting together in the same environment: 

In [10]:
await cog.run_web_ui(
  actions=["no-op", "ArrowUp", "ArrowDown"],
  log_file="human.log",
  fps=30
)

True

In [11]:
trial_id = await cog.start_trial(
    env_name="pong",  # Name of the environment to use
    session_config={"render": True},
    actor_impls={
        "paddle_0": "random",
        "paddle_1": "web_ui"
    },
)
data = await cog.get_trial_data(trial_id)

In [16]:
print(f"Random actions: {data['paddle_0'].actions}")
print(f"Human actions: {data['paddle_1'].actions}")
print(f"Your joint reward: {data['paddle_1'].rewards.sum()}")

Random actions: [0 0 0 2 0 2 1 0 2 0 2 1 2 0 0 2 2 1 0 0 2 2 1 2 1 0 0 2 1 1 2 1 1 0 0 1 2
 1 0 0 0 2 1 2 2 2 1 2]
Human actions: [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 2 2 2 2 2 0 0 0 0 0 1 1 1 1 0 0 2 2
 2 0 0 0 0 0 1 1 1 1 0]
Your joint reward: -4.777777671813965


Feel free to try other environments, play against other agents.

Note: it is currently not possible to have two human "agents" playing at the same time -- it's only one human and an arbitrary number of "AI" agents. This is likely to change in the future.